In [1]:
# загружаем необходимые библиотеки, классы, функции
import numpy as np
import pandas as pd
import dask.dataframe as dd
from sklearn.metrics import roc_auc_score, accuracy_score
from dask_ml.impute import SimpleImputer
from dask_ml.preprocessing import (StandardScaler, 
                                   DummyEncoder, 
                                   Categorizer)
from dask_ml.model_selection import train_test_split
from dask_ml.linear_model import LogisticRegression

In [2]:
# считываем CSV-файл в датафрейм Dask и смотрим
df_dask = dd.read_csv('Data/StateFarm_missing.csv', sep=';')
df_dask

,Customer Lifetime Value,Coverage,Education,EmploymentStatus,Gender,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Response
npartitions=1,,,,,,,,,,,,
,float64,object,object,object,object,float64,float64,float64,float64,float64,float64,object
,...,...,...,...,...,...,...,...,...,...,...,...


In [3]:
# убедимся в том, что работаем с датафреймом Dask
type(df_dask)

dask.dataframe.core.DataFrame

In [4]:
# посмотрим форму датафрейма Dask с помощью 
# функции len() и метода .count()
len(df_dask), len(df_dask.count())

(8293, 12)

In [5]:
# смотрим стастистики, замечаем, что многие переменные
# имеют нормальное распределение (практически одинаковые
# значения средних и медиан)
df_dask.describe().compute()

,Customer Lifetime Value,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies
count,8289.000000,8291.000000,8282.000000,8288.000000,8285.000000,8287.000000,8288.000000
mean,7987.650889,37785.171994,93.198865,15.079875,48.138081,0.389767,2.964768
std,6841.535432,30396.251967,34.514287,10.093847,27.827103,0.915515,2.389635
min,1898.007675,0.000000,61.000000,0.000000,0.000000,0.000000,1.000000
25%,3982.180708,0.000000,68.000000,6.000000,24.000000,0.000000,1.000000
50%,5786.493980,34220.000000,83.000000,14.000000,48.000000,0.000000,2.000000
75%,8960.280213,62450.000000,109.000000,23.000000,71.000000,0.000000,4.000000
max,83325.381190,99981.000000,298.000000,35.000000,99.000000,5.000000,9.000000


In [6]:
# смотрим названия переменных
print(df_dask.columns.tolist())

['Customer Lifetime Value', 'Coverage', 'Education', 'EmploymentStatus', 'Gender', 'Income', 'Monthly Premium Auto', 'Months Since Last Claim', 'Months Since Policy Inception', 'Number of Open Complaints', 'Number of Policies', 'Response']


In [7]:
# смотрим количество пропусков по каждой переменной
df_dask.isnull().compute().sum()

Customer Lifetime Value           4
Coverage                          5
Education                         3
EmploymentStatus                  5
Gender                            4
Income                            2
Monthly Premium Auto             11
Months Since Last Claim           5
Months Since Policy Inception     8
Number of Open Complaints         6
Number of Policies                5
Response                          0
dtype: int64

In [8]:
# смотрим типы переменных
df_dask.dtypes

Customer Lifetime Value          float64
Coverage                          object
Education                         object
EmploymentStatus                  object
Gender                            object
Income                           float64
Monthly Premium Auto             float64
Months Since Last Claim          float64
Months Since Policy Inception    float64
Number of Open Complaints        float64
Number of Policies               float64
Response                          object
dtype: object

In [9]:
# создаем список категориальных переменных
cat_columns = df_dask.select_dtypes(
    include='object').columns.difference(['Response']).tolist()
# смотрим список
cat_columns

['Coverage', 'Education', 'EmploymentStatus', 'Gender']

In [10]:
# смотрим статистики по категориальным переменным
df_dask[cat_columns].describe().compute()

,Coverage,Education,EmploymentStatus,Gender
unique,3,5,5,2
count,8288,8290,8288,8289
top,Basic,Bachelor,Employed,F
freq,5038,2496,5187,4250


In [11]:
# присваиваем переменным типа object тип Categorical
cat = Categorizer(columns=cat_columns)
df_dask = cat.fit_transform(df_dask)

In [12]:
# строковые значения No и Yes переводим 
# в целочисленные 0 и 1
dct = {'No': 0, 'Yes': 1}
df_dask['Response'] = df_dask['Response'].replace(dct)

In [13]:
# выведем частоты категорий по категориальным переменным
for col in cat_columns:
    print(df_dask[col].value_counts().compute())

Basic       5038
Extended    2501
Premium      749
Name: Coverage, dtype: int64
Bachelor                2496
College                 2421
High School or Below    2397
Master                   659
Doctor                   317
Name: Education, dtype: int64
Employed         5187
Unemployed       2095
Medical Leave     392
Disabled          362
Retired           252
Name: EmploymentStatus, dtype: int64
F    4250
M    4039
Name: Gender, dtype: int64


In [14]:
# формируем массив признаков и массив меток
y_dask = df_dask.pop('Response')

In [15]:
# разбиваем данные на обучающие и тестовые: получаем обучающий
# массив признаков, тестовый массив признаков, обучающий массив
# меток, тестовый массив меток
X_train, X_test, y_train, y_test = train_test_split(
    df_dask, 
    y_dask, 
    test_size=0.3,
    shuffle=True,
    random_state=42)

In [16]:
# создаем список количественных столбцов
num_columns = X_train.select_dtypes(
    include='number').columns.tolist()
num_columns

['Customer Lifetime Value',
 'Income',
 'Monthly Premium Auto',
 'Months Since Last Claim',
 'Months Since Policy Inception',
 'Number of Open Complaints',
 'Number of Policies']

In [17]:
# создаем экземпляр класса SimpleImputer
# для количественных переменных
num_imputer = SimpleImputer(strategy='median')
# обучаем модель
num_imputer.fit(X_train[num_columns])
# выполняем импутацию пропусков в количественных переменных
X_train[num_columns] = num_imputer.transform(X_train[num_columns])
X_test[num_columns] = num_imputer.transform(X_test[num_columns])

In [18]:
# создаем экземпляр класса SimpleImputer
# для категориальных переменных
cat_imputer = SimpleImputer(strategy='most_frequent')
# обучаем модель
cat_imputer.fit(X_train[cat_columns])
# выполняем импутацию пропусков в категориальных переменных
X_train[cat_columns] = cat_imputer.transform(X_train[cat_columns])
X_test[cat_columns] = cat_imputer.transform(X_test[cat_columns])

In [19]:
# проверяем наличие пропусков в 
# обучающей и тестовой выборках
print(X_train.isnull().compute().sum().sum())
print(X_test.isnull().compute().sum().sum())

0
0


In [20]:
# создаем экземпляр класса StandardScaler
scaler = StandardScaler()
# обучаем модель
scaler.fit(X_train[num_columns])
# выполняем стандартизацию
X_train[num_columns] = scaler.transform(X_train[num_columns])
X_test[num_columns] = scaler.transform(X_test[num_columns])

In [21]:
# создаем экземпляр класса DummyEncoder
dum = DummyEncoder()
# обучаем модель
dum.fit(X_train)
# выполняем дамми-кодирование
X_train = dum.transform(X_train)
X_test = dum.transform(X_test)

In [22]:
# взглянем на результаты
X_train.head()

,Customer Lifetime Value,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Coverage_Basic,Coverage_Premium,Coverage_Extended,...,Education_Master,Education_High School or Below,Education_Doctor,EmploymentStatus_Employed,EmploymentStatus_Unemployed,EmploymentStatus_Medical Leave,EmploymentStatus_Disabled,EmploymentStatus_Retired,Gender_F,Gender_M
5665,-0.435967,-0.489744,-0.820202,0.493984,0.168462,-0.420026,2.521122,1,0,0,...,0,0,0,0,0,1,0,0,0,1
3273,1.263635,-0.465154,3.462868,1.489298,-1.265742,-0.420026,1.267090,0,0,1,...,0,0,0,1,0,0,0,0,0,1
6804,0.231320,-1.241654,-0.174808,-0.202736,0.993130,-0.420026,-0.404952,0,0,1,...,0,0,0,0,1,0,0,0,1,0
4106,0.172931,0.616429,0.617267,-1.297581,-1.445017,-0.420026,0.013058,1,0,0,...,0,1,0,1,0,0,0,0,1,0
5495,0.025953,0.131054,-0.732194,0.095858,0.921420,-0.420026,-0.404952,1,0,0,...,0,1,0,1,0,0,0,0,1,0


In [23]:
# обучаем модель (для массива Dask с неизвестными размерами чанков 
# строится модель без константы) и оцениваем правильность на
# тестовой выборке с помощью метода .score()
logreg = LogisticRegression(fit_intercept=False, n_jobs=-1)
logreg.fit(X_train.to_dask_array(lengths=True), 
           y_train.to_dask_array(lengths=True))
logreg.score(X_test.to_dask_array(lengths=True), 
             y_test.to_dask_array(lengths=True)).compute()

0.8986623429266315

In [24]:
# создаем массивы Dask с известными размерами чанков
X_train_with_known_chunks = X_train.to_dask_array(lengths=True)
y_train_with_known_chunks = y_train.to_dask_array(lengths=True)
X_test_with_known_chunks = X_test.to_dask_array(lengths=True)
y_test_with_known_chunks = y_test.to_dask_array(lengths=True)

# обучаем модель и оцениваем правильность на тестовой выборке
# с помощью метода .score()
logreg2 = LogisticRegression(fit_intercept=True, n_jobs=-1)
logreg2.fit(X_train_with_known_chunks, y_train_with_known_chunks)
logreg2.score(X_test_with_known_chunks, y_test_with_known_chunks).compute()

0.8986623429266315

In [25]:
# оцениваем правильность с помощью функции accuracy_score
# библиотеки scikit-learn
pred = logreg.predict(X_test.to_dask_array(lengths=True)).compute()
accuracy_score(y_test, pred)

0.8986623429266315

In [26]:
# оцениваем AUC с помощью функции roc_auc_score
# библиотеки scikit-learn
proba = logreg.predict_proba(X_test.to_dask_array(lengths=True)).compute()
roc_auc_score(y_test, proba[:, 1])

0.6010430736905298